In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/uidai-data/api_data_aadhar_enrolment_0_500000.csv
/kaggle/input/uidai-data/api_data_aadhar_enrolment_500000_1000000.csv
/kaggle/input/uidai-data/api_data_aadhar_enrolment_1000000_1006029.csv


In [2]:
import pandas as pd

files = [
    "/kaggle/input/uidai-data/api_data_aadhar_enrolment_0_500000.csv",
    "/kaggle/input/uidai-data/api_data_aadhar_enrolment_500000_1000000.csv",
    "/kaggle/input/uidai-data/api_data_aadhar_enrolment_1000000_1006029.csv"
]

df = pd.concat(
    (pd.read_csv(f) for f in files),
    ignore_index=True
)


In [3]:
df.shape

(1006029, 7)

In [4]:
df.to_csv("uidai_enrolment_full.csv", index=False)


In [5]:
df.head()

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater
0,02-03-2025,Meghalaya,East Khasi Hills,793121,11,61,37
1,09-03-2025,Karnataka,Bengaluru Urban,560043,14,33,39
2,09-03-2025,Uttar Pradesh,Kanpur Nagar,208001,29,82,12
3,09-03-2025,Uttar Pradesh,Aligarh,202133,62,29,15
4,09-03-2025,Karnataka,Bengaluru Urban,560016,14,16,21


In [6]:
df.isnull().sum()

date              0
state             0
district          0
pincode           0
age_0_5           0
age_5_17          0
age_18_greater    0
dtype: int64

In [7]:
df[df.duplicated()].head()

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater
360605,13-10-2025,Punjab,Jalandhar,144041,2,1,0
360606,13-10-2025,Punjab,Jalandhar,144101,1,0,0
360607,13-10-2025,Punjab,Jalandhar,144102,2,0,0
360608,13-10-2025,Punjab,Jalandhar,144418,1,0,0
360609,13-10-2025,Punjab,Jalandhar,144419,1,0,0


In [8]:
df=df.drop_duplicates()

In [9]:
df.duplicated().sum()

np.int64(0)

In [10]:
df.isna().sum()

date              0
state             0
district          0
pincode           0
age_0_5           0
age_5_17          0
age_18_greater    0
dtype: int64

In [11]:

df = df[~((df['state'] == 100000) &
          (df['district'] == 100000) &
          (df['pincode'] == 100000))]


In [12]:
df['state']

0              Meghalaya
1              Karnataka
2          Uttar Pradesh
3          Uttar Pradesh
4              Karnataka
               ...      
1006024      West Bengal
1006025      West Bengal
1006026      West Bengal
1006027      West Bengal
1006028      West Bengal
Name: state, Length: 983072, dtype: object

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 983072 entries, 0 to 1006028
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   date            983072 non-null  object
 1   state           983072 non-null  object
 2   district        983072 non-null  object
 3   pincode         983072 non-null  int64 
 4   age_0_5         983072 non-null  int64 
 5   age_5_17        983072 non-null  int64 
 6   age_18_greater  983072 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 60.0+ MB


In [14]:
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y', errors='coerce')


In [15]:
df = df[df['state'] != '100000']


In [16]:
df['state'] = df['state'].str.lower().str.strip()

In [17]:
state_map = {
    'west bengal': 'west bengal',
    'west bangal': 'west bengal',
    'westbengal': 'west bengal',
    'west  bengal': 'west bengal',
    'odisha': 'odisha',
    'orissa': 'odisha',
    'andhra pradesh': 'andhra pradesh',
    'andhra pradesh': 'andhra pradesh',  # ensure spelling consistent
    'jammu & kashmir': 'jammu and kashmir',
    'jammu and kashmir': 'jammu and kashmir',
    'jammu and kashmir': 'jammu and kashmir',
    'dadra & nagar haveli': 'dadra and nagar haveli',
    'dadra and nagar haveli': 'dadra and nagar haveli',
    'daman & diu': 'daman and diu',
    'daman and diu': 'daman and diu',
    'pondicherry': 'puducherry',
    'andaman & nicobar islands': 'andaman and nicobar islands',
    'andaman and nicobar islands': 'andaman and nicobar islands',
    'westbengal': 'west bengal'
    # add more mappings as needed
}


In [18]:
df['state'] = df['state'].replace(state_map)


In [19]:
df['state'].unique()


array(['meghalaya', 'karnataka', 'uttar pradesh', 'bihar', 'maharashtra',
       'haryana', 'rajasthan', 'punjab', 'delhi', 'madhya pradesh',
       'west bengal', 'assam', 'uttarakhand', 'gujarat', 'andhra pradesh',
       'tamil nadu', 'chhattisgarh', 'jharkhand', 'nagaland', 'manipur',
       'telangana', 'tripura', 'mizoram', 'jammu and kashmir',
       'chandigarh', 'sikkim', 'odisha', 'kerala',
       'the dadra and nagar haveli and daman and diu',
       'arunachal pradesh', 'himachal pradesh', 'goa',
       'dadra and nagar haveli and daman and diu', 'ladakh',
       'andaman and nicobar islands', 'puducherry', 'lakshadweep',
       'dadra and nagar haveli', 'daman and diu'], dtype=object)

In [20]:
# All variations mapped to one canonical UT
dadra_map = {
    'dadra & nagar haveli': 'dadra and nagar haveli and daman and diu',
    'daman & diu': 'dadra and nagar haveli and daman and diu',
    'dadra and nagar haveli': 'dadra and nagar haveli and daman and diu',
    'daman and diu': 'dadra and nagar haveli and daman and diu',
    'dadra and nagar haveli and daman and diu': 'dadra and nagar haveli and daman and diu'
}

# Apply to your state column
df['state'] = df['state'].replace(dadra_map)


In [21]:
# Remove leading "the " from state names
df['state'] = df['state'].str.lower().str.strip().str.replace(r'^the\s+', '', regex=True)

# Now apply your canonical mapping
state_map = {
    'dadra and nagar haveli and daman and diu': 'dadra and nagar haveli and daman and diu',
    # add other mappings if needed
}

df['state'] = df['state'].replace(state_map)


In [22]:
df['district'].nunique()

984

In [23]:
# Remove asterisks and extra spaces
df['district'] = df['district'].str.replace(r'[*]', '', regex=True).str.strip()
